In [2]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy import Request
import time

In [3]:
class TestSpider(scrapy.Spider):
    name = 'test_spider'
    
    rootUrl = 'https://smartstore.naver.com/lingerwater/products/2800118009'    
    start_urls = [rootUrl]
    contents = ''
    
    def start_requests(self):
        headers = {'referer':self.rootUrl}
        cookies = {}        
        pageNum = 2
        numPerPage = 1
        totalPageNum = 0
        sortType = 'REVIEW_RANKING'
        # sortType = 'REVIEW_CREATE_DATE_DESC'
        contentType = 'ALL'
        
        # 처음 total 리뷰수 확인하려면?    
        for i in range(1, pageNum, 1):
            url = 'https://smartstore.naver.com/lingerwater/products/2796364821/reviews/page.json?page={0}&size=30&sortType={1}&contentType={2}'.format(i, sortType, contentType)
            print(url)
            yield Request(url, headers = headers, cookies = cookies, callback = self.parse, method = 'GET')
            time.sleep(1)
            
        
    def parse(self, response):        
        yield print(response.text)
        
        
    


In [4]:
import scrapy.crawler as crawler
from multiprocessing import Process, Queue
from twisted.internet import reactor 

def run_spider(spider):
    def f(q):
        try:
            runner = crawler.CrawlerRunner()
            deferred = runner.crawl(spider)
            deferred.addBoth(lambda _: reactor.stop())
            reactor.run()
            q.put(None)
        except Exception as e:
            q.put(e)
                
    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    result = q.get()
    p.join()
    
    if result is not None:
        raise result

In [5]:
run_spider(TestSpider)

AttributeError: Can't pickle local object 'run_spider.<locals>.f'

In [3]:
# process = scrapy.crawler.CrawlerProcess()
# process.crawl(TestSpider)
# process.start()

In [4]:
from twisted.internet import reactor, defer
from scrapy.crawler import CrawlerRunner
from scrapy.utils.project import get_project_settings

In [9]:
runner = CrawlerRunner(get_project_settings())

@defer.inlineCallbacks
def crawl(spider):
    yield runner.crawl(spider)
    reactor.stop()

In [11]:
#reactor.stop()
crawl(TestSpider)
reactor.run()

ReactorNotRestartable: 